In [1]:
import pickle

In [2]:
import pandas as pd
GROUPS = None
#GROUPS = 20

df = pd.read_pickle("experiment-multi_dim_multivariate-2018-07-23-013012.pkl")['benchmark_results']
df = df.append(pd.read_pickle("experiment-run-1-1043199673-2018-08-01-194308.pkl")['benchmark_results'])
df = df.append(pd.read_pickle("experiment-run-0-906869411-2018-08-01-131414.pkl")['benchmark_results'])
df = df.groupby(["dataset", "algorithm"], as_index=False).mean()
df = df[["dataset", "algorithm", "auroc"]]
df.head()


/home/madness/Documents/Projects/MP-2018/venv3.6_prebuilt_mp/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


,dataset,algorithm,auroc
0,Synthetic Multivariate 1000-dimensional extrem...,DAGMM_LSTMAutoEncoder_withWindow,0.721979
1,Synthetic Multivariate 1000-dimensional extrem...,DAGMM_NNAutoEncoder_withWindow,0.709913
2,Synthetic Multivariate 1000-dimensional extrem...,DAGMM_NNAutoEncoder_withoutWindow,0.617652
3,Synthetic Multivariate 1000-dimensional extrem...,LSTM-AD,0.867358
4,Synthetic Multivariate 1000-dimensional extrem...,LSTMED,0.912554


In [3]:
if GROUPS is None:
    df = df[~df['dataset'].str.contains("with 20 ")]
else:
    df = df[df['dataset'].str.contains("with 20 ")]

In [4]:
import collections

def recursively_default_dict():
    return collections.defaultdict(recursively_default_dict)

results = recursively_default_dict() # function - algorithm - dimension
algorithm_name = {'DAGMM_LSTMAutoEncoder_withWindow': 'DAGMM-LW', 'DAGMM_NNAutoEncoder_withWindow': 'DAGMM-NW',
                  'DAGMM_NNAutoEncoder_withoutWindow': 'DAGMM-NN', 'LSTM-AD': 'LSTM-AD', 'Recurrent EBM': 'REBM', 'LSTMED': 'LSTMED'}
dimensions = set()
functions = ['Doubled', 'Shrinked', 'Inversed', 'XOR', 'Delayed', 'Delayed Missing']
algorithms = set()

for _, row in df.iterrows():
    function =  row[0].split('(f=')[1].split(')')[0]
    dimension = int(row[0].split('-dimensional')[0].split(' ')[2])
    dimensions.add(dimension)
    algorithm = algorithm_name[row[1]]
    algorithms.add(algorithm)
    score = float(row[2])
    results[function][algorithm][dimension] = score
algorithms = sorted(list(algorithms))
results['delayed']

defaultdict(<function __main__.recursively_default_dict()>,
            {'DAGMM-LW': defaultdict(<function __main__.recursively_default_dict()>,
                         {1000: 0.7219789005591151,
                          200: 0.8286239971093473,
                          500: 0.7143285684447318}),
             'DAGMM-NW': defaultdict(<function __main__.recursively_default_dict()>,
                         {1000: 0.7099129521725381,
                          200: 0.8064511680740732,
                          500: 0.6732210412839976}),
             'DAGMM-NN': defaultdict(<function __main__.recursively_default_dict()>,
                         {1000: 0.6176518068898726,
                          200: 0.7989211838404136,
                          500: 0.7093837759347876}),
             'LSTM-AD': defaultdict(<function __main__.recursively_default_dict()>,
                         {1000: 0.8673577003212409,
                          200: 0.9558321578104841,
                          500:

In [5]:
for function_index, function in enumerate(functions):
    function_lower = function.lower().replace(' ', '_')
    title = f"({function_index}) {function}"
    x_label = "Dimensions"
    x_max = str(max(dimensions) + 100)
    
    # Add figure block
    header = ""
    if function_index == 0:
        header += "\\begin{figure}[!h]\n\\centering"
    header += "" + \
    """\\begin{subfigure}{.475\\linewidth}
    \\centering
    """
    
    header += u"""
    \\begin{tikzpicture}[yscale=0.9]
    \\begin{axis}[
        title={{the_title}},
        xlabel={x_label},
        ylabel={AUROC},
        xmin=0, xmax={x_max},
        ymin=0, ymax=1,
        xtick={dimensions},
        ytick={0,0.2, 0.4, 0.6, 0.8, 1.0},
        legend pos=north west,
        ymajorgrids=true,
        grid style=dashed,
        legend style ={ at={(0,0)}, 
            anchor=south west, draw=black, 
            fill=white,align=left},
        %cycle list name=black white,
        width=\\linewidth
    ]
    """.replace("the_title", title).replace("x_label", x_label).replace("dimensions",",".join([str(x) for x in sorted(list(dimensions))])).replace("x_max", x_max)
    print(header)
    for algorithm in algorithms:
        add_plot = """
        \\addplot
            coordinates {
            """
        for dimension in dimensions:
            add_plot += f"({dimension},{results[function_lower][algorithm][dimension]})"
        print(add_plot + "};")
    if function_index == 0:
        print("\\legend{algorithms}".replace("algorithms", ",".join(algorithms)))
    print("""
    \\end{axis}
    \\end{tikzpicture}
    \\end{subfigure}""")
    if function_index % 2 == 0:
        print("\\hfill")
    else:
        if function_index == len(functions)-1:
            group_text = "The group size is set to "
            if GROUPS is None:
                group_text += "the number of dimensions."
            else:
                group_text += "20."
            
            print("\\caption{Comparison of the presented approaches on different anomalies with varying dimensions. " + group_text + "}")
            if GROUPS is None:
                print("\\label{fig:multid_groups_none}")
            else:
                print("\\label{fig:multid_groups_20}")
            print("\\end{figure}")

\begin{figure}[!h]
\centering\begin{subfigure}{.475\linewidth}
    \centering
    
    \begin{tikzpicture}[yscale=0.9]
    \begin{axis}[
        title={{(0) Doubled}},
        xlabel={Dimensions},
        ylabel={AUROC},
        xmin=0, xmax={1100},
        ymin=0, ymax=1,
        xtick={200,500,1000},
        ytick={0,0.2, 0.4, 0.6, 0.8, 1.0},
        legend pos=north west,
        ymajorgrids=true,
        grid style=dashed,
        legend style ={ at={(0,0)}, 
            anchor=south west, draw=black, 
            fill=white,align=left},
        %cycle list name=black white,
        width=\linewidth
    ]
    

        \addplot
            coordinates {
            (1000,0.8838631027866177)(500,0.8729175464339403)(200,0.8945548894735972)};

        \addplot
            coordinates {
            (1000,0.7682799296259315)(500,0.8943649339101674)(200,0.9315798541741227)};

        \addplot
            coordinates {
            (1000,0.7181955368745607)(500,0.7074390000165067)(200,0.91